In [8]:
import csv
import json
import requests
from ipwhois import IPWhois

In [9]:
def read_file(filename):
    output = []
    with open(filename) as f:
        data = f.readlines()
        output = [x.strip() for x in data]
    return output

In [10]:
IP_address_file = "./../Processed Data/IP_Address_list.txt"
AS_number_file = "./../Processed Data/AS_Number_list.txt"

ip_address_list = read_file(IP_address_file)
as_number_list = read_file(AS_number_file)

In [11]:
def get_geolocation(ip_address):
    request_url = 'https://geolocation-db.com/jsonp/' + ip_address
    response = requests.get(request_url)
    result = response.content.decode()
    result = result.split("(")[1].strip(")")
    result  = json.loads(result)
    return result

In [21]:
data = []
data_1 = []

for i, ip in enumerate(ip_address_list):
    if as_number_list[i] == "-100" or as_number_list == 'None':
        continue
    else:
        result = get_geolocation(ip)
        city = result['city']
        country_code = result['country_code']
        latitude = result['latitude']
        longitude = result['longitude']
        
        whois_result = IPWhois(ip).lookup_whois()
        ip_range = whois_result['nets'][0]['range']
        description = whois_result['nets'][0]['description']
        
        data.append([as_number_list[i], ip, city, country_code, latitude, longitude])
        data_1.append([as_number_list[i], ip_range, city, country_code, description])

In [22]:
fields = ['AS Number', 'IP Address', 'City', 'Country Code', 'Latitude', 'Longitude']
csv_filename = "./../Processed Data/AS_Number_IP_Address_Country_Code.csv"

with open(csv_filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(data)

In [24]:
fields = ['AS Number', 'IP Range', 'City', 'Country Code', 'Description']
csv_filename = "./../Processed Data/IP_address_range.csv"

with open(csv_filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(data_1)